In [ ]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import DensityMatrix
from qiskit_aer.quantum_info import AerDensityMatrix
from qiskit.circuit.library import QuantumVolume

In [ ]:
qc = QuantumVolume(5, seed=1111)
%timeit state = DensityMatrix(qc)

In [ ]:
qc = QuantumVolume(5, seed=1111)
%timeit state = AerDensityMatrix(qc)

In [ ]:
qc = QuantumVolume(7, seed=1111)
%timeit state = DensityMatrix(qc)

In [ ]:
qc = QuantumVolume(7, seed=1111)
%timeit state = AerDensityMatrix(qc)

In [ ]:
qc = QuantumVolume(20, seed=1111)
%timeit state = DensityMatrix(qc)

In [ ]:
qc = QuantumVolume(20, seed=1111)
%timeit state = AerDensityMatrix(qc)

In [ ]:
qc = QuantumVolume(21, seed=1111)
%timeit state = DensityMatrix(qc)

In [ ]:
qc = QuantumVolume(21, seed=1111)
%timeit state = AerDensityMatrix(qc)